In [1]:
import sys
sys.path.append(rf"/home/a/Lwy_research")
from Functions import *

In [2]:
# 这一部分函数之前有
# 计算完整的秒序列
def calc_sec_list(keep_last = False):
    time_series=[]  #所有的秒序列,不包括早尾盘集合竞价(也就是093000-145659的)
    for i in [9,10,11,13,14]:
        for j in range(60):
            if i==9 and j<30:
                continue
            elif i==11 and j>=30:
                continue
            elif i==14 and j>=57:
                continue
            time_series.extend([int(str(i)+'0'*(2-len(str(j)))+str(j)+'0'*(2-len(str(x)))+str(x)) for x in range(60)])
    if keep_last: # 保留最后一个时点
        time_series += [150000]
    return time_series

# 转化为1秒级时间,比如如果time为93524380，且转化的1s时间为93524000
def get_second_1s(time_10ms):
    div = 1000
    return round(round(time_10ms // div) * div) # 先取整再标准化为Level2时间

# 转化为分钟时间,比如如果time为93524380，且转化的1s时间为93500000
def get_second_60s(time_10ms):
    div = 100000
    return round(round(time_10ms // div) * div) # 先取整再标准化为Level2时间

In [3]:
# 对一些基本的逐笔成交、逐笔委托数据进行计算
def calc_stock_second_data_basic(code, trans_data, order_data,  intra_support_data, standard_time_list, get_second_map):
    '''
    使用逐笔成交和逐笔委托构成的合成数据，包含一些统计量，如过去几秒的订单量、价格等
    要求尽可能使用流式计算，加快速度，如均值、标准差等统计量都可以一笔订单一笔订单计算
    '''
# try:
    def _process_second_data0(code, second_data0, standard_time_list, intra_support_data):
        second_data0 = second_data0.reindex(standard_time_list)
        second_data = pd.DataFrame().reindex_like(second_data0)
        pre_close = intra_support_data.loc[code, 'preclose']
        preprocess_dict = {
            1: ['OrderPriceMean_B','OrderPriceStd_B','OrderPriceMean_S','OrderPriceStd_S',
                'TradePriceMean_B','TradePriceStd_B','TradePriceMean_S','TradePriceStd_S',
                'Open','High','Low','Close', 
                'OrderAmtSum_B','OrderAmtMax_B','OrderAmtbig_B','OrderAmtmid_B','OrderAmtsma_B',
                'OrderAmtSum_S','OrderAmtMax_S','OrderAmtbig_S','OrderAmtmid_S','OrderAmtsma_S',
                'TradeAmtSum_B','TradeAmtMax_B','TradeAmtbig_B','TradeAmtmid_B','TradeAmtsma_B',
                'TradeAmtSum_S','TradeAmtMax_S','TradeAmtbig_S','TradeAmtmid_S','TradeAmtsma_S',
                "CancelAmtSum_B", "CancelAmtMax_B", "CancelAmtSum_S", "CancelAmtMax_S",
                ],
            2: ['OrderVolSum_B','OrderVolMax_B','OrderVolbig_B','OrderVolmid_B','OrderVolsma_B','OrderVolSquareSum_B',
                'OrderVolSum_S','OrderVolMax_S','OrderVolbig_S','OrderVolmid_S','OrderVolsma_S','OrderVolSquareSum_S',
                'CancelVolSum_B','CancelVolMax_B','CancelVolSquareSum_B','CancelVolSumJg_B','CancelVolSumPart_B',
                'CancelVolSum_S','CancelVolMax_S','CancelVolSquareSum_S','CancelVolSumJg_S','CancelVolSumPart_S',
                'TradeVolSum_B','TradeVolMax_B','TradeVolbig_B','TradeVolmid_B','TradeVolsma_B','TradeVolSquareSum_B',
                'TradeVolSum_S','TradeVolMax_S','TradeVolbig_S','TradeVolmid_S','TradeVolsma_S','TradeVolSquareSum_S',
                'OrderNumSum_B','OrderNumSum_S','CancelNumSum_B','CancelNumSum_S','TradeNumSum_B','TradeNumSum_S'
                ],
        }
        
        for field in second_data0.columns:
            df = second_data0[field]
            # 对基础数据进行标准化，价格和成交金额相关的除以preclose，成交额和笔数相关的不用
            if field in preprocess_dict[1]:
                df = df / pre_close
            elif field in preprocess_dict[2]:
                df = df
            else:
                raise ValueError(f"Field not in preprocess_dict: {field}")
            
            if (field in ["Open", "High", "Low", "Close"]) : # 价格填充
                df = df.replace(0,np.nan).ffill().fillna(1.0)
            elif ('Price' in field): # 涉及到价格数据，统一不进行填充，保留nan
                pass
            else: # 量额填充
                df = df.fillna(0.0)
            second_data[field] = df
        return second_data
    
    temp_list = [] # 将所有的初步结果拼接起来的列表
    # 逐笔成交数据处理
    trans_data["second"] = trans_data["time"].map(get_second_map)
    trans_data['transVolSquare']=trans_data['transVolume']**2/10000
    
    # 成交订单在计算大小单阈值时不区分B、S
    # 计算早盘成交金额数据，用来衡量大中小单, 意味着10点之前衡量每笔订单大小的阈值会不一样，10点之后订单的判断阈值统一按照10点的组来定
    trans_data["tradeAmtMean"] = trans_data.rolling(len(trans_data), min_periods = 1)["transAmount"].mean().values
    if len(trans_data) == 1:
        trans_data["tradeAmtStd"] = 0
    else:
        trans_data["tradeAmtStd"] = trans_data.rolling(len(trans_data), min_periods = 2)["transAmount"].std().values
    trans_data.loc[trans_data["time"] >= normalize_time("1000"), ["tradeAmtMean", "tradeAmtStd"]] = np.nan
    trans_data[["tradeAmtMean", "tradeAmtStd"]] = trans_data[["tradeAmtMean", "tradeAmtStd"]].ffill()
    trans_data["AmtType"] = "mid"
    trans_data.loc[trans_data["transAmount"] <= trans_data["tradeAmtMean"], "AmtType"] = "sma"
    trans_data.loc[trans_data["transAmount"] >= (trans_data["tradeAmtMean"] +  trans_data["tradeAmtStd"]), "AmtType"] = "big" 
    
    # 再处理成交部分，分买卖
    for side in ["B", "S"]:
        data = trans_data[trans_data["transSide"]==side]
        if len(data) == 0:
            continue
        #计算一些统计量指标
        data_group = data.groupby(["second"])
        temp1 = data_group.agg({"transPrice": ["count", "mean", "std"],
                                "transVolume": ["sum", "max"],
                                "transAmount": ["sum", "max"],
                                "transVolSquare": ["sum"]})
        
        temp1.columns = ["TradeNumSum","TradePriceMean" ,"TradePriceStd", "TradeVolSum", "TradeVolMax", "TradeAmtSum", "TradeAmtMax",'TradeVolSquareSum']    
        temp2 = data.groupby(["second", "AmtType"])["transAmount"].sum().unstack()
        temp2.columns = ["TradeAmt" + x  for x in temp2.columns]
        temp3 = data.groupby(["second", "AmtType"])["transVolume"].sum().unstack()
        temp3.columns = ["TradeVol" + x  for x in temp3.columns]
        temp = pd.concat([temp1, temp2, temp3], axis=1)
        temp.columns = [col+"_%s" % side for col in temp.columns]
        temp_list.append(temp)

    # 最后计算每个second开高低收价格
    data = trans_data.copy()
    temp = data.groupby(["second"]).agg({"transPrice": ["first", "max", "min", "last"]})
    temp.columns = ["Open", "High", "Low", "Close"]
    temp_list.append(temp)

    # 逐笔委托和撤单数据处理
    order_data["second"] = order_data["time"].map(get_second_map)
    # 1457之后的委托归属150000000的second
    order_data.loc[order_data.second >= 145700000, "second"] = 150000000
    
    # 在计算基础字段时，撤单和委托数据的成交量、金额都取成正的
    order_data[["orderVolume", "orderAmount"]] = np.abs(order_data[["orderVolume", "orderAmount"]])
    
    order_data['orderVolSquare']=order_data['orderVolume']**2/10000
    cancel_data = order_data[order_data["orderType"].isin(['C','D'])] # 撤单数据
    order_data = order_data[~order_data["orderType"].isin(['C','D'])] # 委托数据
    
    
    # 计算一笔订单撤单时间和原始委托时间差
    cancel_data = cancel_data.merge(order_data[['orderIdOrigin','time','orderVolume']],on=['orderIdOrigin'],how='left')
    cancel_data=cancel_data.rename(columns={'time_x':'time',
                                            'orderVolume_x':'orderVolume'})
    cancel_data['interval']=(pd.to_datetime(cancel_data['time'],format='%H%M%S%f')- \
                                pd.to_datetime(cancel_data['time_y'],format='%H%M%S%f')).dt.seconds

    #撤单数据目前保存在委托数据中
    for side in ["B", "S"]:
        data = cancel_data[cancel_data["orderSide"]==side]
        if len(data) == 0:
            continue

        data_group = data.groupby(["second"])
        temp1 = data_group.agg({"orderVolume": ["count", "sum", "max"],
                                "orderAmount": ["sum", "max"],
                                "orderVolSquare":["sum"]})
        temp1.columns = ["CancelNumSum", "CancelVolSum", "CancelVolMax","CancelAmtSum", "CancelAmtMax", "CancelVolSquareSum"]
        
        data1 = data[data['interval']<15]  #撤单时间小于15s
        temp2 = data1.groupby(["second"]).agg({"orderVolume": ["sum"]})
        temp2.columns= ['CancelVolSumJg']
        
        data2=data[-data['orderVolume']+data['orderVolume_y']>0]  #撤单量小于委托量的
        temp3=data2.groupby(["second"]).agg({"orderVolume": ["sum"]})
        temp3.columns= ['CancelVolSumPart']
        
        temp=pd.concat([temp1, temp2, temp3], axis=1)
        temp.columns = [col+"_%s" % side for col in temp.columns]
        temp_list.append(temp)

    order_data  = order_data.sort_values(["time", "orderId"]).reset_index(drop = True)
    for side in ["B", "S"]:
        data = order_data[order_data["orderSide"]==side]
        data2 = data.loc[data["orderType"].isin(['1','A'])] # 仅含限价单
        if len(data2) == 0:
            continue
        # 计算早盘成交金额数据，用来衡量大中小单, 意味着10点之前衡量每笔订单大小的阈值会不一样，10点之后订单的判断阈值统一按照10点的组来定
        data2["orderAmtMean"] = data2.rolling(len(data2), min_periods = 1)["orderAmount"].mean().values
        if len(data2) == 1:
            data2["orderAmtStd"] = 0
        else:
            data2["orderAmtStd"] = data2.rolling(len(data2), min_periods = 2)["orderAmount"].std().values
        data2.loc[data2["time"] >= normalize_time("1000"), ["orderAmtMean", "orderAmtStd"]] = np.nan
        data2[["orderAmtMean", "orderAmtStd"]] = data2[["orderAmtMean", "orderAmtStd"]].ffill()
        
        data2["AmtType"] = "mid"
        data2.loc[data2["orderAmount"] <= data2["orderAmtMean"], "AmtType"] = "sma"
        data2.loc[data2["orderAmount"] >= (data2["orderAmtMean"] +  data2["orderAmtStd"]), "AmtType"] = "big" 
        
        temp1 = data2.groupby(["second"]).agg({"orderPrice": ["count", "mean", "std"],
                                "orderVolume": ["sum", "max"],
                                "orderAmount": ["sum", "max"],
                                "orderVolSquare": ["sum"]})
        temp1.columns = ["OrderNumSum", "OrderPriceMean","OrderPriceStd", "OrderVolSum", "OrderVolMax","OrderAmtSum", "OrderAmtMax","OrderVolSquareSum"]
        temp2 = data2.groupby(["second", "AmtType"])["orderAmount"].sum().unstack()
        temp2.columns = ["OrderAmt" + x  for x in temp2.columns]
        temp3 = data2.groupby(["second", "AmtType"])["orderVolume"].sum().unstack()
        temp3.columns = ["OrderVol" + x  for x in temp3.columns]
        temp = pd.concat([temp1, temp2, temp3], axis=1)
        temp.columns = [col+"_%s" % side for col in temp.columns]
        temp_list.append(temp)


    # 进一步处理秒级数据
    second_data0 = pd.concat(temp_list, axis=1)
    second_data = _process_second_data0(code, second_data0, standard_time_list, intra_support_data)
    second_data["Code"] = code
    second_data = second_data.reset_index().set_index("Code")
    return second_data
# except Exception as e:
#     print(code,e)
#     return pd.DataFrame()

In [ ]:
time_step = "60s" # 这里time_step有可能是1s、5s、10s、60s，这里用60s只作为示例和核对作用
get_second_map = eval(rf"get_second_{time_step}")
second_list = calc_sec_list(keep_last=True)
second_list = [x*1000 for x in second_list]
standard_time_list = []
for time in tqdm(second_list):
    if get_second_map(time) not in standard_time_list:
        standard_time_list.append(get_second_map(time))
len(standard_time_list)

100%|██████████| 14221/14221 [00:00<00:00, 1089904.38it/s]


238

In [5]:
for date in get_trade_days("20240910", "20240913"):
    sample_order_data = pd.read_feather(rf"./example_data/sample_order_{date}.fea")
    sample_trans_data = pd.read_feather(rf"./example_data/sample_trans_{date}.fea")
    order_data_dict = change_df2dict(sample_order_data)
    trans_data_dict = change_df2dict(sample_trans_data)
    intra_support_data = pd.read_csv(rf"./example_data/stock_intra_support_{date}.csv", index_col="Code")
    intra_support_data.index = ['%06d'%int(x) for x in intra_support_data.index]

    code_list = set(order_data_dict.keys()) & set(order_data_dict.keys())

    # for code in tqdm(code_list):
    #     order_data = order_data_dict[code]
    #     trans_data = trans_data_dict[code]
    #     second_data = calc_stock_second_data(code, trans_data, order_data,  intra_support_data, standard_time_list, get_second_map)

    second_data_list = Parallel(n_jobs=32)(delayed(calc_stock_second_data_basic)(code, trans_data_dict[code], order_data_dict[code],  intra_support_data, standard_time_list, get_second_map) for code in tqdm(code_list))
    pd.concat(second_data_list).reset_index().to_feather(rf"./example_data/sample_second_{date}.fea")

100%|██████████| 1000/1000 [00:07<00:00, 125.35it/s]
